# Example: Dynamic Delta Neutral Hedging to Compensate for Price Fluctuations
[Delta hedging](https://en.wikipedia.org/wiki/Delta_neutral) is a strategy used in options trading to minimize the risk associated with price movements in the underlying asset. This approach involves using other options contracts or shares of the underlying asset to achieve a delta-neutral position, i.e., eliminating the directional bias of the position.
* __Scenario 1__: You are long a `TSLA` put option (thus, you are short the shares) and are concerned about how the price movement of `TSLA` shares influences the overall risk of your position. Toward this risk, let's explore constructing a delta-neutral hedge.
* __Scenario 2__: You are short a `TSLA` call option (thus, you are also short the shares) and are concerned about how the price movement of `TSLA` shares influences the overall risk of your position. Toward this risk, let's explore constructing a delta-neutral hedge.

Both scenarios (s) 1 and 2 make the same directional assumption, namely, that `TSLA` shares will decline in price, i.e., you are `bearish` on `TSLA.` Let's explore how delta-neutral hedging would work for these scenarios.

## Learning objectives
Fill me in.

## Setup
Set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, various functions we will use in the exercise, and custom types to model the components of our example problem.
* For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 

In [3]:
include("Include.jl")

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-4/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-4/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-4`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-4/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-4/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-134/module-4/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CH

## Prerequisites

Fill me in

In [33]:
option_dataset = load(joinpath(_PATH_TO_DATA, "TSLA-today-2023-06-19-exp-2023-08-18-dte-60d.jld2")) |> file -> file["data"];

One-aggregate data for `TSLA` for 2023

In [41]:
underlying_price_data_df = CSV.read(joinpath(_PATH_TO_DATA, "TSLA-OHLC-1-min-aggregate-2023.csv"), DataFrame);

Compute the vwap, and the log_growth_matrix 

In [44]:
vwap(underlying_price_data_df) |> data -> log_growth_matrix(data, Δt 

Row,date,open,high,low,close,volume
,String31,Float64,Float64,Float64,Float64,Int64
1,2023-01-03 09:30:00,118.47,118.8,116.98,117.09,2774007
2,2023-01-03 09:31:00,117.1,117.17,116.23,116.49,1396098
3,2023-01-03 09:32:00,116.5,116.7,115.33,115.79,1447550
4,2023-01-03 09:33:00,115.74,116.39,115.47,116.31,938296
5,2023-01-03 09:34:00,116.27,116.86,115.52,115.72,997526
6,2023-01-03 09:35:00,115.74,115.95,114.7,115.164,1460946
7,2023-01-03 09:36:00,115.13,115.24,114.14,114.219,1064546
8,2023-01-03 09:37:00,114.22,114.53,113.89,114.388,1313213
9,2023-01-03 09:38:00,114.4,114.59,113.721,114.0,1078696


## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. You should decide solely based on your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.